In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps
from math import sqrt

plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 11


# 1. Babyboom — сравнение среднего и дисперсий между девочками и мальчиками

Задачи:
- Проверить H0: средний вес девочек = средний вес мальчиков.
- Проверить H0: дисперсия веса девочек = дисперсия веса мальчиков.

Подход:
- Оценка нормальности (Shapiro, Q-Q) для ориентировки.
- Тест на равенство дисперсий: Levene (робастный) и классический F-тест (информативно).
- Для средних: t-test (equal_var=True при равных дисперсиях, иначе Welch с equal_var=False).


In [2]:
babyboomDataFrame = pd.read_csv('datasets/babyboom.dat.txt', sep='\s+', header=None,
    names=['timeOfBirth24', 'sexCode', 'birthWeightGrams', 'minutesAfterMidnight'])
babyboomDataFrame['sexCode'] = babyboomDataFrame['sexCode'].astype(int)
babyboomDataFrame['birthWeightGrams'] = pd.to_numeric(babyboomDataFrame['birthWeightGrams'], errors='coerce')

print('Размер выборки (всего):', len(babyboomDataFrame))
display(babyboomDataFrame.head())


Размер выборки (всего): 44


,timeOfBirth24,sexCode,birthWeightGrams,minutesAfterMidnight
0,5,1,3837,5
1,104,1,3334,64
2,118,2,3554,78
3,155,2,3838,115
4,257,2,3625,177


In [3]:
weightSeries = babyboomDataFrame['birthWeightGrams'].dropna()
girlsSeries = babyboomDataFrame.loc[babyboomDataFrame['sexCode']==1, 'birthWeightGrams'].dropna()
boysSeries = babyboomDataFrame.loc[babyboomDataFrame['sexCode']==2, 'birthWeightGrams'].dropna()

def printSummary(series, label):
    print(f"\n--- {label} ---")
    print('N =', series.size)
    print('Mean =', series.mean())
    print('SD =', series.std(ddof=1))
    
    try:
        if series.size >= 3:
            shStat, shP = sps.shapiro(series.sample(min(5000, series.size)))
            print('Shapiro-Wilk statistic =', round(shStat,6), ', p-value =', round(shP,6))
    except Exception as e:
        print('Shapiro error:', e)
    
    try:
        muHat = series.mean()
        sigmaHat = series.std(ddof=1)
        ksStat, ksP = sps.kstest(series, 'norm', args=(muHat, sigmaHat))
        print('Kolmogorov-Smirnov statistic =', round(ksStat,6), ', p-value =', round(ksP,6))
    except Exception as e:
        print('KS test error:', e)

printSummary(weightSeries, 'All')
printSummary(girlsSeries, 'Girls (sexCode=1)')
printSummary(boysSeries, 'Boys (sexCode=2)')



--- All ---
N = 44
Mean = 3275.9545454545455
SD = 528.0324582405921
Shapiro-Wilk statistic = 0.898723 , p-value = 0.000994
Kolmogorov-Smirnov statistic = 0.183364 , p-value = 0.091065

--- Girls (sexCode=1) ---
N = 18
Mean = 3132.4444444444443
SD = 631.582533586883
Shapiro-Wilk statistic = 0.870283 , p-value = 0.017985
Kolmogorov-Smirnov statistic = 0.214278 , p-value = 0.331584

--- Boys (sexCode=2) ---
N = 26
Mean = 3375.3076923076924
SD = 428.0460507217203
Shapiro-Wilk statistic = 0.947474 , p-value = 0.202248
Kolmogorov-Smirnov statistic = 0.155443 , p-value = 0.507076


In [4]:
# Тесты на равенство дисперсий
levStatistic, levPvalue = sps.levene(girlsSeries, boysSeries, center='median')
print('Levene: statistic =', round(levStatistic,4), ', p-value =', round(levPvalue,6))

#F-тест (двухсторонний)
varGirls = girlsSeries.var(ddof=1)
varBoys = boysSeries.var(ddof=1)
fStatistic = varGirls / varBoys if varBoys>0 else np.nan
df1 = girlsSeries.size - 1
df2 = boysSeries.size - 1
if not np.isnan(fStatistic):
    if fStatistic >= 1:
        fPvalue = 2 * (1 - sps.f.cdf(fStatistic, df1, df2))
    else:
        fPvalue = 2 * sps.f.cdf(fStatistic, df1, df2)
else:
    fPvalue = np.nan

print('F-test (variance ratio): F =', round(fStatistic,4), 'df1=', df1, 'df2=', df2, ', p-value =', fPvalue)

Levene: statistic = 1.8154 , p-value = 0.185085
F-test (variance ratio): F = 2.1771 df1= 17 df2= 25 , p-value = 0.07526261914285004


In [5]:
# Тест на равенство средних
equalVarChoice = (levPvalue >= 0.05) # если Levene не отклоняет равенство дисперсий, используем equal_var=True
print('Выбор equal_var для t-test =', equalVarChoice)

tStatistic, tPvalueTwoSided = sps.ttest_ind(girlsSeries, boysSeries, equal_var=equalVarChoice)
print('Two-sample t-test (two-sided): t =', round(tStatistic,4), ', p-value =', round(tPvalueTwoSided,6))

tPvalueOneSided = tPvalueTwoSided / 2
print('Two-sided p-value =', tPvalueTwoSided, ', one-sided p-value (meanGirls < meanBoys) =', round(tPvalueOneSided,6))


Выбор equal_var для t-test = True
Two-sample t-test (two-sided): t = -1.5229 , p-value = 0.135289
Two-sided p-value = 0.1352891891054555 , one-sided p-value (meanGirls < meanBoys) = 0.067645


# 2. Euroweight — равенство средних между пакетами (batch)

Задачи:
- Проверить, одинаковы ли средние веса монет во всех пакетах (ANOVA).
- Провести попарные сравнения между пакетами (попарные t-тесты с поправкой Bonferroni).


In [6]:
euroweightDataFrame = pd.read_csv('datasets/euroweight.dat.txt', sep='\s+', header=None, names=['index','weight','batch'])
euroweightDataFrame['weight'] = pd.to_numeric(euroweightDataFrame['weight'], errors='coerce')
euroweightDataFrame['batch'] = euroweightDataFrame['batch'].astype(int)

display(euroweightDataFrame.head())

groupsList = [group['weight'].dropna().values for name, group in euroweightDataFrame.groupby('batch')]
fStatisticAnova, pValueAnova = sps.f_oneway(*groupsList)
print('ANOVA: F =', round(fStatisticAnova,4), ', p-value =', round(pValueAnova,6))


,index,weight,batch
0,1,7.512,1
1,2,7.502,1
2,3,7.461,1
3,4,7.562,1
4,5,7.528,1


ANOVA: F = 12.6722 , p-value = 0.0


In [7]:
# Попарные сравнения (Welch t-test) и поправка Bonferroni
from itertools import combinations
batchGroups = {batchId: group['weight'].dropna().values for batchId, group in euroweightDataFrame.groupby('batch')}
batchIds = sorted(batchGroups.keys())
pairwiseResults = []
for a, b in combinations(batchIds, 2):
 seriesA = batchGroups[a]
 seriesB = batchGroups[b]
 tStat, pVal = sps.ttest_ind(seriesA, seriesB, equal_var=False)
 pairwiseResults.append({'batchA': a, 'batchB': b, 'tStatistic': tStat, 'pValue': pVal})

numComparisons = len(pairwiseResults)
for res in pairwiseResults:
 res['pValueBonferroni'] = min(res['pValue'] * numComparisons, 1.0)

print('Pairwise comparisons (Welch), Bonferroni-corrected p-values:')
for res in pairwiseResults:
 print(f"{res['batchA']} vs {res['batchB']}: t={res['tStatistic']:.4f}, p={res['pValue']:.6f}, p_bonf={res['pValueBonferroni']:.6f}")


Pairwise comparisons (Welch), Bonferroni-corrected p-values:
1 vs 2: t=-1.1242, p=0.261478, p_bonf=1.000000
1 vs 3: t=3.1645, p=0.001649, p_bonf=0.046173
1 vs 4: t=-4.0017, p=0.000073, p_bonf=0.002035
1 vs 5: t=-4.0915, p=0.000050, p_bonf=0.001404
1 vs 6: t=1.4566, p=0.145861, p_bonf=1.000000
1 vs 7: t=-1.1152, p=0.265323, p_bonf=1.000000
1 vs 8: t=0.9227, p=0.356621, p_bonf=1.000000
2 vs 3: t=4.1995, p=0.000032, p_bonf=0.000888
2 vs 4: t=-2.7223, p=0.006718, p_bonf=0.188101
2 vs 5: t=-2.8143, p=0.005088, p_bonf=0.142454
2 vs 6: t=2.5714, p=0.010418, p_bonf=0.291704
2 vs 7: t=0.0496, p=0.960464, p_bonf=1.000000
2 vs 8: t=2.0013, p=0.045904, p_bonf=1.000000
3 vs 4: t=-7.2069, p=0.000000, p_bonf=0.000000
3 vs 5: t=-7.2845, p=0.000000, p_bonf=0.000000
3 vs 6: t=-1.8051, p=0.071663, p_bonf=1.000000
3 vs 7: t=-4.2939, p=0.000021, p_bonf=0.000592
3 vs 8: t=-2.1904, p=0.028956, p_bonf=0.810771
4 vs 5: t=-0.1106, p=0.911988, p_bonf=1.000000
4 vs 6: t=5.6335, p=0.000000, p_bonf=0.000001
4 vs 7:

# 3. Iris — равенство распределений, средних и дисперсий по видам

Задачи:
- Проверить равенство распределений характеристик между типами ирисов.
- Проверить равенство средних (ANOVA) и равенство дисперсий (Levene) для каждой характеристики.


In [8]:
irisDataFrame = pd.read_csv('datasets/iris.txt', header=None, names=['sepalLength','sepalWidth','petalLength','petalWidth','class'])
irisDataFrame['sepalLength'] = pd.to_numeric(irisDataFrame['sepalLength'], errors='coerce')
irisDataFrame['sepalWidth'] = pd.to_numeric(irisDataFrame['sepalWidth'], errors='coerce')
irisDataFrame['petalLength'] = pd.to_numeric(irisDataFrame['petalLength'], errors='coerce')
irisDataFrame['petalWidth'] = pd.to_numeric(irisDataFrame['petalWidth'], errors='coerce')

featuresList = ['sepalLength','sepalWidth','petalLength','petalWidth']
classLabels = sorted(irisDataFrame['class'].unique())
print('Classes:', classLabels)

from itertools import combinations

for feature in featuresList:
    print('\n=== Feature:', feature, '===')
    groups = [group[feature].dropna().values for name, group in irisDataFrame.groupby('class')]
    kwStat, kwP = sps.kruskal(*groups)
    print('Kruskal-Wallis: H =', round(kwStat,4), ', p-value =', round(kwP,6))
    
    fStat, fP = sps.f_oneway(*groups)
    print('ANOVA (means): F =', round(fStat,4), ', p-value =', round(fP,6))
    
    levStat, levP = sps.levene(*groups, center='median')
    print('Levene (variances): stat =', round(levStat,4), ', p-value =', round(levP,6))
    
    print('Pairwise KS tests:')
    for a, b in combinations(classLabels, 2):
        seriesA = irisDataFrame.loc[irisDataFrame['class']==a, feature].dropna()
        seriesB = irisDataFrame.loc[irisDataFrame['class']==b, feature].dropna()
        ksStat, ksP = sps.ks_2samp(seriesA, seriesB)
        print(f"{a} vs {b}: KS stat={ksStat:.4f}, p={ksP:.6f}")

    print('Pairwise t-tests (Welch) with Bonferroni:')
    pairwisePvals = []
    pairwisePairs = []
    for a, b in combinations(classLabels, 2):
        seriesA = irisDataFrame.loc[irisDataFrame['class']==a, feature].dropna()
        seriesB = irisDataFrame.loc[irisDataFrame['class']==b, feature].dropna()
        tS, pV = sps.ttest_ind(seriesA, seriesB, equal_var=False)
        pairwisePairs.append((a,b))
        pairwisePvals.append(pV)
        correctedPvals = [min(p * len(pairwisePvals), 1.0) for p in pairwisePvals]
    for (a,b), rawP, corrP in zip(pairwisePairs, pairwisePvals, correctedPvals):
        print(f"{a} vs {b}: t p={rawP:.6f}, p_bonf={corrP:.6f}")


Classes: ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

=== Feature: sepalLength ===
Kruskal-Wallis: H = 96.9374 , p-value = 0.0
ANOVA (means): F = 119.2645 , p-value = 0.0
Levene (variances): stat = 6.3527 , p-value = 0.002259
Pairwise KS tests:
Iris-setosa vs Iris-versicolor: KS stat=0.7800, p=0.000000
Iris-setosa vs Iris-virginica: KS stat=0.9200, p=0.000000
Iris-versicolor vs Iris-virginica: KS stat=0.4600, p=0.000038
Pairwise t-tests (Welch) with Bonferroni:
Iris-setosa vs Iris-versicolor: t p=0.000000, p_bonf=0.000000
Iris-setosa vs Iris-virginica: t p=0.000000, p_bonf=0.000000
Iris-versicolor vs Iris-virginica: t p=0.000000, p_bonf=0.000001

=== Feature: sepalWidth ===
Kruskal-Wallis: H = 62.4946 , p-value = 0.0
ANOVA (means): F = 47.3645 , p-value = 0.0
Levene (variances): stat = 0.6475 , p-value = 0.524827
Pairwise KS tests:
Iris-setosa vs Iris-versicolor: KS stat=0.6800, p=0.000000
Iris-setosa vs Iris-virginica: KS stat=0.5000, p=0.000005
Iris-versicolor vs Iris-virgin

# 4. Surgery — проверка вероятности успешной операции (p = 0.7 и p = 0.8)

Условие успеха: у пациента success = (V_right_after > V_right_before) и (V_left_after > V_left_before).

Будем использовать точный биномиальный тест и z-тест для пропорции.


In [9]:
surgeryDataFrame = pd.read_excel('datasets/surgery.xlsx', header=1)

display(surgeryDataFrame.head())

surgeryDataFrame.rename(columns={
    surgeryDataFrame.columns[0]: 'beforeRight',
    surgeryDataFrame.columns[1]: 'beforeLeft',
    surgeryDataFrame.columns[2]: 'afterRight',
    surgeryDataFrame.columns[3]: 'afterLeft'
}, inplace=True)

def toFloatSafe(x):
    try:
        if pd.isna(x):
            return np.nan
        return float(str(x).replace(',', '.'))
    except:
        return np.nan

for col in ['beforeRight','beforeLeft','afterRight','afterLeft']:
    surgeryDataFrame[col + 'Val'] = surgeryDataFrame[col].apply(toFloatSafe)

surgeryClean = surgeryDataFrame.dropna(subset=['beforeRightVal','beforeLeftVal','afterRightVal','afterLeftVal']).copy()

surgeryClean['success'] = (
    (surgeryClean['afterRightVal'] > surgeryClean['beforeRightVal']) &
    (surgeryClean['afterLeftVal'] > surgeryClean['beforeLeftVal'])
)

numPatients = surgeryClean.shape[0]
numSuccesses = int(surgeryClean['success'].sum())

print('Количество пациентов с полными данными =', numPatients)
print('Количество успехов =', numSuccesses)


,V right,V left,V right.1,V left.1
0,7.2,6.70,12.0,13.1
1,1.2,1.20,4.5,4.2
2,6.7,7.30,15.3,14.9
3,9.9,10.05,9.6,9.1
4,3.1,2.13,NaN,NaN


Количество пациентов с полными данными = 87
Количество успехов = 69


In [ ]:
def runBinomialTests(successes, n, pHypothesisList=[0.7, 0.8]):
    for pHyp in pHypothesisList:
        print(f'--- H0: p = {pHyp} ---')
        try:
            testResult = sps.binomtest(successes, n, p=pHyp)
            pValue = testResult.pvalue
            print('Binomtest (two-sided) p-value =', pValue)
        except Exception as e:
            print('binomtest error:', e)
 
        propHat = successes / n if n>0 else np.nan
        stdUnderH0 = sqrt(pHyp * (1 - pHyp) / n) if n>0 else np.nan
        if stdUnderH0 and stdUnderH0>0:
            zStat = (propHat - pHyp) / stdUnderH0
            pZTwoSided = 2 * (1 - sps.norm.cdf(abs(zStat)))
            print('Prop z-test: z =', round(zStat,4), ', two-sided p-value =', round(pZTwoSided,6))
        else:
            print('Prop z-test: stdUnderH0 == 0 или n==0, нельзя вычислить')
        print('')

runBinomialTests(numSuccesses, numPatients, [0.7, 0.8])

--- H0: p = 0.7 ---
Binomtest (two-sided) p-value = 0.061288170927676786
Prop z-test: z = 1.895 , two-sided p-value = 0.058089

--- H0: p = 0.8 ---
Binomtest (two-sided) p-value = 0.8933020521355081
Prop z-test: z = -0.1608 , two-sided p-value = 0.872238

